<a href="https://colab.research.google.com/github/YoonTae-Hwang/trading_repository/blob/master/1_Backtrader_Qucistart_Guide_ANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- ANS 챕터에서는 각 코드 블록 하나하나씩을 리뷰하는 시간을 갖도록 하겠습니다.

In [1]:
from google.colab import files
upload = files.upload()

for i in upload.keys():
  print("User uploaded file  {name} with length {length} bytes".format(name = i, length = len(upload[i])))

Saving BTC-USD.csv to BTC-USD.csv
User uploaded file  BTC-USD.csv with length 146474 bytes


In [49]:
import pandas as pd
import io

BTC = pd.read_csv(io.StringIO(upload["BTC-USD.csv"].decode("utf-8")),
                  skiprows = 0,
                  parse_dates = True,
                  index_col = 0)
BTC

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-04-10,243.694000,243.694000,232.770996,236.072006,236.072006,2.888200e+07
2015-04-11,236.016006,239.537003,234.175003,236.552002,236.552002,1.636520e+07
2015-04-12,236.535004,237.727997,233.494995,236.153000,236.153000,1.238790e+07
2015-04-13,235.949997,236.934998,221.996002,224.587006,224.587006,3.118180e+07
2015-04-14,224.759003,224.975998,216.322998,219.158997,219.158997,3.171900e+07
...,...,...,...,...,...,...
2020-04-06,6788.049805,7271.781250,6782.889648,7271.781250,7271.781250,4.689690e+10
2020-04-07,7273.644043,7427.939453,7136.714355,7176.414551,7176.414551,4.424348e+10
2020-04-08,7179.283203,7356.223633,7153.305664,7334.098633,7334.098633,3.756325e+10


- backtrader 의 동작과정은 다음과 같습니다.
   1. Start by creating a __bt.Cerebro()__
   2. cerebro.addstrategy(name)
   3. Add data feed : data feed과정에서는 __quandl, pandas, yahoo 등 다양한 데이터__를 사용할 수 있습니다.
   4. Set initial cash : __cerebro.broker.setcash()__
   5. Run cerebro : cerebro.run()

In [13]:
!pip install backtrader
import matplotlib.pyplot as plt
import backtrader as bt
import datetime
import os.path
import sys

     |████████████████████████████████| 419kB 2.7MB/s 


In [50]:
if __name__ == "__main__":
  cerebro = bt.Cerebro()
  
  cerebro.addstrategy(TestStrategy)

  data = bt.feeds.PandasData(dataname = BTC)
  cerebro.adddata(data)

  cerebro.broker.setcash(15000.0)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  
  cerebro.run()

  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 15000.00
2015-04-10, Close, 236.07
2015-04-11, Close, 236.55
2015-04-12, Close, 236.15
2015-04-13, Close, 224.59
2015-04-14, Close, 219.16
2015-04-15, Close, 223.83
2015-04-16, Close, 228.57
2015-04-17, Close, 222.88
2015-04-18, Close, 223.36
2015-04-19, Close, 222.60
2015-04-20, Close, 224.63
2015-04-21, Close, 235.27
2015-04-22, Close, 234.18
2015-04-23, Close, 236.46
2015-04-24, Close, 231.27
2015-04-25, Close, 226.39
2015-04-26, Close, 219.43
2015-04-27, Close, 229.29
2015-04-28, Close, 225.85
2015-04-29, Close, 225.81
2015-04-30, Close, 236.15
2015-05-01, Close, 232.08
2015-05-02, Close, 234.93
2015-05-03, Close, 240.36
2015-05-04, Close, 239.02
2015-05-05, Close, 236.12
2015-05-06, Close, 229.78
2015-05-07, Close, 237.33
2015-05-08, Close, 243.86
2015-05-09, Close, 241.83
2015-05-10, Close, 240.30
2015-05-11, Close, 242.16
2015-05-12, Close, 241.11
2015-05-13, Close, 236.38
2015-05-14, Close, 236.93
2015-05-15, Close, 237.60
2015-05-16, Close, 236.15
201

- 전략을 세우기 위해서는 다음과 같이 class를 만든다. 첫번째 블럭을 살펴보자.

1. ``` python
  def log(self, txt, dt = None):
    dt = dt or self.datas[0].datetime.date(0)
    print("%s, %s" % (dt.isoformat(), txt))
```
- log function은 우리가 전략을 실행하였을때 출력되는 문구이다.
- dt는 datetime을 의미하고, isoformat을 함으로써 ISO 8601 형식으로 날짜를 나타내는 문자열을 반환한다. 
- txt의 경우에는 현재 주가데이터가 close인지, open인지 나타낸다. 일반적으로 closed를 사용한다.

***

2. ``` python
    def __init__(self):
        self.dataclose = self.datas[0].close
```

- Initialization function을 살펴보자. 여기에서는 다양한 incidators들을 넣는다. (RSI, Momentom, DMA ... etc)
- 위의 예제에서는 특정한 incidators를 사용하지 않고 당일 close만 받기로하자.

***


3. ``` python
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
```

- 다음으로 next function을 살펴보자. next함수에서는 Initialization function에서의 결과와 현재의 closed 값을 비교해 주식 구매, 판매, 유지, 취소 즉, 다음과 같은 행동 buy/ sell/ close(exisiting position) / cancel (cancel a not yet executed order)을 수행한다. 
- 위의 예시에서는 매일매일의 close 값들만 출력이 됨을 알 수 있다. 

In [0]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])